[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Machine Learning - Supervised Learning - Ensemble Methods - MNIST by 1D Features

The notebook is based on [Aaron Zuspan - Classifying MNIST as 1D Signals](https://www.aazuspan.dev/blog/classifying-mnist-as-1d-signals).

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 25/08/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0002PointLine.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Scientific Python

# Image Processing & Computer Vision
import skimage as ski

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Miscellaneous
import math
from platform import python_version
import random

# Typing 
from typing import Callable, List, Optional, Tuple
from numpy.typing import NDArray

# Visualization
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ?????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

TU_MNIST_IMG_SIZE = (28, 28)

In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions

def ConvertDataSet( mX: NDArray, /, *, resampleRes: Optional[int] = None ) -> NDArray:
    # mX: (numSamples, 768) MNIST like Array
    numSamples = np.size(mX, 0)

    tI = np.reshape(mX, (-1, 28, 28)) #<! The last dimension is contiguous
    tI = np.transpose(tI, (1, 2, 0))

    if resampleRes is not None:
        tI = ski.transform.resize(tI, (resampleRes, resampleRes))
    
    tP = ski.transform.warp_polar(tI, channel_axis = 2) #<! Polar (360, radius)
    mP = np.sum(tP, axis = 1) #<! (360, numSamples)

    return mP.T #<! Return a matrix of (numSamples, numFeatures)

def PlotMnistImages( mX: np.ndarray, vY: np.ndarray, numRows: int, numCols: Optional[int] = None, tuImgSize: Tuple = (28, 28), randomChoice: bool = True, lClasses: Optional[List] = None, hF: Optional[plt.Figure] = None ) -> plt.Figure:

    numSamples  = mX.shape[0]
    numPx       = mX.shape[1]

    if numCols is None:
        numCols = numRows

    tFigSize = (numCols * 3, numRows * 3)

    if hF is None:
        hF, hA = plt.subplots(numRows, numCols, figsize = tFigSize)
    else:
        hA = hF.axes
    
    hA = np.atleast_1d(hA) #<! To support numImg = 1
    hA = hA.flat
    
    for kk in range(numRows * numCols):
        idx = np.random.choice(numSamples) if randomChoice else kk
        mI  = np.reshape(mX[idx, :], tuImgSize)
    
        # hA[kk].imshow(mI.clip(0, 1), cmap = 'gray')
        if len(tuImgSize) == 2:
            hA[kk].imshow(mI, cmap = 'gray')
        elif len(tuImgSize) == 3:
            hA[kk].imshow(mI)
        else:
            raise ValueError(f'The length of the image size tuple is {len(tuImgSize)} which is not supported')
        hA[kk].tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                           labelleft = False, labeltop = False, labelright = False, labelbottom = False)
        if lClasses is None:
            hA[kk].set_title(f'Index = {idx}, Label = {vY[idx]}')
        else:
            hA[kk].set_title(f'Index = {idx}, Label = {lClasses[vY[idx]]}')
    
    return hF

def PlotLabelsHistogram( vY: np.ndarray, hA: Optional[plt.Axes] = None, lClass: Optional[List] = None, xLabelRot: Optional[int] = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))
    
    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_xticks(vLabels, [f'{labelVal}' for labelVal in vLabels])
    hA.set_ylabel('Count')
    if lClass is not None:
        hA.set_xticklabels(lClass)
    
    if xLabelRot is not None:
        for xLabel in hA.get_xticklabels():
            xLabel.set_rotation(xLabelRot)

    return hA

## 1D Features for 1D Signal Classification

fd

### The MNIST Dataset



* <font color='red'>(**?**)</font> Will the solution ofr the Squared Euclidean Distance be the same as the Euclidean Distance?

In [ ]:
# Parameters

numSamplesTrain = 9_000
numSamplesTest  = 1_000

numImg = 3

# Visualization
exportFig = False

## Generate Data


In [ ]:
# Generate / Load Data 

mX, vY = fetch_openml('mnist_784', version = 1, return_X_y = True, as_frame = False, parser = 'auto')
vY = vY.astype(np.int_) #<! The labels are strings, convert to integer

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')
print(f'The unique values of the labels: {np.unique(vY)}')

In [ ]:
# Pre Processing

# The image is in the range {0, 1, ..., 255}
# We scale it into [0, 1]

#===========================Fill This===========================#
# 1. Scale the values into the [0, 1] range.
mX = mX / 255.0

#===============================================================#

In [ ]:
# Train Test Split

#===========================Fill This===========================#
# 1. Split the data such that the Train Data has `numSamplesTrain`.
# 2. Split the data such that the Test Data has `numSamplesTest`.
# 3. The distribution of the classes must match the original data.

numClass = len(np.unique(vY))
mXTrain, mXTest, vYTrain, vYTest = train_test_split(mX, vY, test_size = numSamplesTest, train_size = numSamplesTrain, shuffle = True, stratify = vY)

#===============================================================#

print(f'The training features data shape: {mXTrain.shape}')
print(f'The training labels data shape  : {vYTrain.shape}')
print(f'The test features data shape    : {mXTest.shape}')
print(f'The test labels data shape      : {vYTest.shape}')
print(f'The unique values of the labels : {np.unique(vY)}')

### Explore the Data

In [ ]:
# Plot the Data

hF = PlotMnistImages(mX, vY, numImg)

In [ ]:
# Distribution of Labels

hA = PlotLabelsHistogram(vY)
plt.show()

In [ ]:
# Mean Image per Class

tI = np.zeros(shape = (numClass, ) + TU_MNIST_IMG_SIZE)

for ii in range(numClass):
    vIdx = vY == ii
    vF = np.mean(mX[vIdx], axis = 0) #<! (numFeatures, )
    tI[ii] = np.reshape(vF, TU_MNIST_IMG_SIZE)

In [ ]:
# Plot Mean Images

hF, vHa = plt.subplots(nrows = 1, ncols = numClass, figsize = (18, 2))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    hA.imshow(tI[ii], cmap = 'gray')
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                   labelleft = False, labeltop = False, labelright = False, labelbottom = False)
    hA.set_title(f'Label {ii}')

hF.suptitle('Mean Class Image');

### Cartesian and Polar Coordinate Systems

Main motivation is to transform rotations into translations.

In [ ]:
# Cartesian and Polar Coordinates for an Image

hF = plt.figure(figsize = (8, 4))

hAxCart  = hF.add_subplot(1, 2, 1)
hAxPolar = hF.add_subplot(1, 2, 2, projection = 'polar')

# Cartesian Grid
Nx, Ny = 8, 8                   # grid size
hAxCart.set_xlim(0, Nx)
hAxCart.set_ylim(Ny, 0)         # y grows downward (like images)
hAxCart.set_aspect('equal')

# Grid
for x in range(Nx + 1):
    hAxCart.axvline(x, color = '0.75', lw = 1)
for y in range(Ny + 1):
    hAxCart.axhline(y, color = '0.75', lw = 1)

# Image in (x, y) Discrete Coordinates (1 based)
# Highlight pixel (6, 3) 
px, py = 6, 3
patchRect = Rectangle((px - 1, py - 1), 1, 1, facecolor = '0.7', edgecolor = 'k')
hAxCart.add_patch(patchRect)

# Annotations
hAxCart.annotate('Pixel (6, 3)', xy = (px - 0.5, py - 0.5),
                 xytext = (px - 2.5, py + 1.3),
                 arrowprops = dict(arrowstyle = "->", lw = 1.2),
                 ha = 'center', va = 'center')

hAxCart.set_title('Cartesian Coordinates')
hAxCart.set_xlabel('x')
hAxCart.set_xticks(range(Nx + 1))
hAxCart.set_xticklabels([])
hAxCart.set_ylabel('y')
hAxCart.set_yticks(range(Ny + 1))
hAxCart.set_yticklabels([])

# Polar Grid
nr, nt = 6, 24 #<! Radial and Angular Grid resolution
R = nr         #<! Outer radius
hAxPolar.set_ylim(0, R)
hAxPolar.set_thetalim(0, 2 * math.pi)

# Grid: radial circles and spokes
hAxPolar.set_rticks(range(1, nr + 1))
hAxPolar.set_thetagrids(np.degrees(np.linspace(0, 2 * math.pi, nt, endpoint = False)))
hAxPolar.grid(True, lw = 0.8, color = '0.75')

# Image in (r, θ) Discrete Coordinates (1 based)
# Highlight sector / pixel (2, 4)
ri, ti = 2, 4
dr = R / nr
dth = 2 * math.pi / nt
theta0 = (ti - 1) * dth
bottom = (ri - 1) * dr
# Use a polar bar to draw the annular sector
hAxPolar.bar(theta0, dr, width = dth, bottom = bottom, align = 'edge',
             color = '0.7', edgecolor = 'k')

hAxPolar.set_title('Polar Coordinates')
hAxPolar.annotate(f'Pixel ({ri}, {ti})',
                  xy = (theta0 + dth / 2, bottom + dr / 2),
                  xytext = (theta0 + 1.1 * dth, bottom + 2.2 * dr),
                  arrowprops = dict(arrowstyle = "->", lw = 1.2),
                  ha = 'center')

hAxPolar.set_yticklabels([]) 

hF.tight_layout()

if exportFig:
    hF.savefig('Coordinates.svg', transparent = True)

<!-- Should include the ExcaliDraw embedded in the image -->
![](https://i.imgur.com/hh5Hnhv.png)
<!-- ![](https://i.postimg.cc/9FP5HDQ9/Untitled-2025-05-03-2057-excalidraw.png) -->

### Aggregation per Angle

Summing pixels along the radial axis provides a profile of of the radial distribution around the image center.

In [ ]:
# Polar and Aggregation Representation per Image

rndIdx = random.randrange(numSamplesTrain)
mI = np.reshape(mXTrain[rndIdx], TU_MNIST_IMG_SIZE)
mP = ski.transform.warp_polar(mI, output_shape = (280, 280))
vP = np.sum(mP, axis = 1)

In [ ]:
# Plot Polar and Aggregation Representation per Image 

hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 3))
vHa = vHa.flat

hA = vHa[0]
hA.imshow(mI, cmap = 'gray')
hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
               labelleft = False, labeltop = False, labelright = False, labelbottom = False)
hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_title('Cartesian Coordinates')

hA = vHa[1]
hA.imshow(mP.T[::-1], cmap = 'gray')
hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
               labelleft = False, labeltop = False, labelright = False, labelbottom = False)
hA.set_xlabel('θ')
hA.set_ylabel('r')
hA.set_title('Polar Coordinates')

hA = vHa[2]
hA.plot(vP)
hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
               labelleft = False, labeltop = False, labelright = False, labelbottom = False)
hA.set_xlabel('θ')
hA.set_ylabel('Sum of Values')
hA.set_title('Aggregation per θ');

In [ ]:
# Plot Animation
mP = ski.transform.warp_polar(mI, output_shape = (360, 360))
vP = np.sum(mP, axis = 1)


hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 4))
vHa = vHa.flat

tuCenter  = (TU_MNIST_IMG_SIZE[1] // 2, TU_MNIST_IMG_SIZE[0] // 2)
valRadius = math.sqrt(2) * max(tuCenter)

hA = vHa[0]
hA.imshow(mI, cmap = 'gray')
lineAngle, *_ = hA.plot([tuCenter[0], tuCenter[0] + valRadius], [tuCenter[1], tuCenter[1]], color = 'r', lw = 2)
hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
               labelleft = False, labeltop = False, labelright = False, labelbottom = False)
hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_xlim((0, TU_MNIST_IMG_SIZE[0] - 1))
hA.set_ylim((TU_MNIST_IMG_SIZE[1] - 1, 0))
hA.set_title(f'θ = {0:03d} [Deg]')

hA = vHa[1]
hA.plot(vP)
lineSum = hA.axvline(x = 0, color = 'r', lw = 2)
hA.set_xlim((0, 360))

figName = f'Figure{0:04d}.png'
if exportFig:
    hF.savefig(figName, dpi = 150)

for ii, θ in enumerate(range(360)):
    θRad = -math.radians(θ)  #<! Convert degrees to radians
    xEnd = tuCenter[0] + valRadius * math.cos(θRad)
    yEnd = tuCenter[1] - valRadius * math.sin(θRad) #<! Inverted as Y direction is down
    lineAngle.set_data([tuCenter[0], xEnd], [tuCenter[1], yEnd])
    lineSum.set_xdata([θ])
    vHa[0].set_title(f'θ = {θ:03d} [Deg]')
    
    hF.canvas.draw() #<! Update the canvas before exporting

    figName = f'Figure{(ii + 1):04d}.png'
    if exportFig:
        hF.savefig(figName, dpi = 150)
# ffmpeg -framerate 10 -i Figure%04d.png -c:v libx264 -pix_fmt yuv420p -an -movflags faststart -loop 1 output.mp4

### Projection Visualization

<iframe width="853" height="480" src="//sendvid.com/embed/kh2a1tox" frameborder="0" allowfullscreen></iframe>
<!-- <iframe allow="fullscreen" allowfullscreen height="480" src="https://streamable.com/e/g1g71s?" width="800" style="border:none;"></iframe> -->

### Analysis of the 1D Signals

In [ ]:
# Transform Data into 1D
mPTrain = ConvertDataSet(mXTrain, resampleRes = 56)

In [ ]:
# Mean Curve per Class

mM = np.zeros(shape = (numClass, mPTrain.shape[1]))

for ii in range(numClass):
    vIdx = vYTrain == ii
    vF = np.mean(mPTrain[vIdx], axis = 0) #<! (360, )
    mM[ii] = vF

In [ ]:
# Plot the Mean Curve per Class

hF, vHa = plt.subplots(nrows = 2, ncols = numClass // 2, figsize = (12, 4))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    vIdx = np.flatnonzero(vYTrain == ii)
    vIdx = np.random.choice(vIdx, size = min(25, len(vIdx)), replace = False)
    mL = mPTrain[vIdx]
    hA.plot(mL.T, lw = 0.5, color = 'k', alpha = 0.3)
    hA.plot(mM[ii], lw = 2)
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                   labelleft = False, labeltop = False, labelright = False, labelbottom = False)
    hA.set_title(f'Label {ii}')

hF.suptitle('Mean Class Curve');

In [ ]:
# Draw the 

In [ ]:
# Generate the Lines
# Lines parameters: a x + b y + c = 0

mL = np.array([[1, 1, 0], [-0.8, 1.7, -0.7], [-0.8, 1.6, -0.75], [1.1, -0.95, -0.05]]) #<! Lines parameters: a, b, c
numLines = mL.shape[0]

#===========================Fill This===========================#
# 1. Normalize the lines so that a^2 + b^2 = 1.
# !! Make sure the line is the same.

vN = np.linalg.norm(mL[:, :2], axis = 1) #<! Calculate the norm of (a, b)
mL /= vN[:, None]                        #<! Scales all parameters

#===============================================================#

In [ ]:
# Convert the Lines into the y = m x + n Form

# The motivation is for visualization
mM = np.column_stack((-mL[:, 0] / mL[:, 1], -mL[:, 2] / mL[:, 1])) #<! Not safe, in production verify `mL[:, 1] != 0`

* <font color='red'>(**?**)</font> If all lines are valid functions, is the above operation safe? Consider in theory and in practice.

In [ ]:
# Draw the Data

hF, hA = plt.subplots(figsize = (10, 7))

for ii in range(numLines):
    vY = np.array([mM[ii, 0] * valX + mM[ii, 1] for valX in vX])
    hA.plot(vX, vY, lw = LINE_WIDTH_DEF, label = f'Line {ii + 1:02d}')

hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_title('Set of Lines')

hA.legend();

## Deriving the Objective Function

This section formulates the problem as a _Quadratic Programming_ problem.

### Distance of a Point from a Line

Given a point $\boldsymbol{x} = \left[ {x}_{0}, {y}_{0} \right]^{T}$ then the _Euclidean Distance_ to a line, given by the parameters $l = \left( a, b, c \right) : a x + b y + c = 0$ is given by:

$$ d \left( \boldsymbol{x}, l \right) = \frac{ \left| a {x}_{0} + b {y}_{0} + c \right| }{ \sqrt{ {a}^{2} + {b}^{2} } } $$

Assuming all lines are normalized, then the operation becomes:

$$ d \left( \boldsymbol{x}, l \right) = a {x}_{0} + b {y}_{0} + c $$

The objective function, based on the _Squared Euclidean Distance_, for the DF as defined is given by:

$$ \arg \min_{\boldsymbol{x} \in \mathbb{R}^{2}} \sum_{i = 1}^{n} {d}^{2} \left( \boldsymbol{x}, \boldsymbol{l}_{i} \right) $$


* <font color='brown'>(**#**)</font> Using the Squared Euclidean as a measure of distance greatly simplifies the derivation of the solution.

### Simplified Form

Since a line is defined by $l = \left( a, b, c \right) : a x + b y + c = 0$, multiplying its parameters by a constant has no effect.  
Namely, lines are parameterized up to a scale. It means one can chose to have the line in a form which ${a}^{2} + {b}^{2} = 1$.

Given that, the optimization problem is given by:

$$ \arg \min_{\boldsymbol{x} \in \mathbb{R}^{2}} \sum_{i = 1}^{n} {d}^{2} \left( \boldsymbol{x}, \boldsymbol{l}_{i} \right) = \arg \min_{\boldsymbol{x} \in \mathbb{R}^{2}} \sum_{i = 1}^{n} {\left( {a}_{i} x + {b}_{i} y + {c}_{i} \right)}^{2} $$

* <font color='brown'>(**#**)</font> This form can be farther simplified so it can be solved as _Linear Least Squares_ problem.

In [ ]:
# Define the Objective Function

def PointLineSumSqrDistance( vX: np.ndarray, mL: np.ndarray ) -> np.floating:
    """
    The function calculates the sum of squared distances of a point in 2D to a set of lines.
    For a line, `vL`, a row in the matrix `mL`:
    The line is given as `vL[0] * x + vL[1] * y + vL[2] = 0`.  
    The line is assumed to have `np.linalg.norm(vL[:2]) = 1`.
    Input:
        vX      - A point in 2D (2, ).
        mL      - Set of 2D lines parameters (numLines x 3).
    Output:
        sumDisSqr - The sum of squared distance to lines.
    """
    #===========================Fill This===========================#
    # 1. Set `sumDisSqr` to zero.
    # 2. Loop over lines and add the distance to the line.
    # !! Output must be a scalar.
    # !! You may use a vectorized method.
    
    sumDisSqr = 0.0
    for vL in mL:
        sumDisSqr += anp.sum(anp.square(vL[0] * vX[0] + vL[1] * vX[1] + vL[2])) #<! Add the distance to the line
    
    # Vectorized Method
    # sumDisSqr = anp.sum(anp.square(mL[:, 0] * vX[0] + mL[:, 1] * vX[1] + mL[:, 2])) #<! Add the distance to the line
    #===============================================================#
    
    return sumDisSqr

In [ ]:
# Find the Optimal Point

def FindOptimalPoint( vX: np.ndarray, hGradFun: Callable, /, *, μ: float = 1e-5, numIter: int = 10_000 ) -> np.ndarray:
    """
    The function finds the optimal point that minimizes the sum of squared distances to a set of lines.
    Input:
        vX       - The initial point in 2D (2, ).
        hGradFun - The gradient function of the objective function: `hGradFun(vX) -> vG`.
        μ        - The step size for the gradient descent.
        numIter  - The number of iterations for the gradient descent.
    Output:
        vX      - The optimal point in 2D (2, ).
    Remarks:
        - The function assumes the arrays `vX` and `mL` are AutoGrad arrays.
    """
    #===========================Fill This===========================#
    # 1. Create Gradient Descent loop.
    # 2. Calculate the gradient using `hGradFun`.
    # !! You may add a stopping condition.
    # !! You may add adaptive step size.

    for _ in range(numIter):
        vG = hGradFun(vX) #<! Calculate the gradient using `hGradFun()`
        vX -= μ * vG #<! Update the point using gradient descent
    #===============================================================#
    
    return vX

* <font color='brown'>(**#**)</font> Production level implementation should add: Adaptive Step Size, Acceleration Method, Stopping Condition, Flag for Convergence.


In [ ]:
# Find the Optimal Point Using Gradient Descent

hObjFun  = lambda vX: PointLineSumSqrDistance(vX, mL) #<! Define the objective function
hGradFun = grad(hObjFun) #<! Define the gradient function using `AutoGrad`
vP       = anp.zeros(2) #<! Initial point

vP = FindOptimalPoint(vP, hGradFun, μ = μ, numIter = numIter) #<! Find the point

* <font color='green'>(**@**)</font> Replace `FindOptimalPoint()` with `sp.optimize.fmin_bfgs()` and compare run time.

<!-- vP = anp.zeros(2)
vP = sp.optimize.fmin_bfgs(hObjFun, vP, hGradFun) -->

In [ ]:
# Plot the Optimal Solution 

hF, hA = plt.subplots(figsize = (10, 7))

for ii in range(numLines):
    vY = np.array([mM[ii, 0] * valX + mM[ii, 1] for valX in vX])
    hA.plot(vX, vY, lw = LINE_WIDTH_DEF, label = f'Line {ii + 1:02d}')

hA.scatter(vP[0], vP[1], s = 4 * MARKER_SIZE_DEF, color = '#FFE119', label = 'Optimal Point', zorder = 2.1)

hA.set_aspect('equal') #<! Must in order to have 90 [Deg]
hA.set_xlabel('x')
hA.set_ylabel('y')
hA.set_title('Set of Lines and the Optimal Point')

hA.legend();

* <font color='green'>(**@**)</font> Create _Heatmap_ of the objective function.
* <font color='green'>(**@**)</font> Create _Heatmap_ of the _Euclidean Distance_ and compare.
* <font color='red'>(**?**)</font> In what cases the Squared Euclidean Distance will fail?